In [2]:
import numpy as np
import os
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle

In [4]:
classes = os.listdir('../DataSets/data_set/')
print classes

['dogs', 'cats', 'Humans', 'horses']


In [6]:
dict_labels = {'dogs': 0, 'cats': 1, 'Humans' : 2, 'horses': 3}

In [ ]:
x_, y_ = []
for ix in classes:
    imgdir = os.listdir()